#The original data set consists of 12*12 files, one for each month from 2011-2022
#Using those directly in SQL was not possible.
#To simplify my queries, I combined the files into 12, one for each year.
#To find find the best way regarding performance and storage size, I tried different methods.


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import os
import glob, os
import pandas as pd
import polars as pl

In [ ]:
# At first I used Pandas to transform each parquet into a CSV file.
#I tried this, to have CSVs to import into SQL. Two issues arose:
#The combined files where 6 times larger than the actual parquet files
#And the import into MYSQL turned out to be too slow.
#Even with a reading speed of 10.000 rows per minute, it would have taken months to import 1 Billion rows.


months=range(1,13)
years=range(2011,2023)

for y in years:
    for x in months:
        if x>9:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-{month}.parquet".format(year=y,month=x)

            parq= pd.read_parquet(path)
            parq.to_csv(r"/content/drive/MyDrive/Taxitaxi/CSVs/{year}/yellow_tripdata_{year}-{month}".format(year=y,month=x))

        else:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-0{month}.parquet".format(year=y,month=x)
            parq= pd.read_parquet(path)
            parq.to_csv(r"/content/drive/MyDrive/Taxitaxi/CSVs/{year}/yellow_tripdata_{year}-0{month}".format(year=y,month=x))

        print(y, x)

print("fertig")

In [ ]:
#I started combining the files into 12 bigger ones, to work with them directly in Python
#Here I used Pandas agaiin, to concatinate and save them as parquets.
#I already excluded multiple columns, which where of little use
#Still, Pandas is just made for data, that is much smaller than the available RAM


print("start")


columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime',  'trip_distance',
'PULocationID','DOLocationID','total_amount','tip_amount']

months=range(1,13)
years=range(2011,2023)

for y in years:
    for x in months:
        if x==1:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-0{month}.parquet".format(year=y,month=x)
            combined= pd.read_parquet(path,columns=columns)
        elif x>9:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-{month}.parquet".format(year=y,month=x)
            parq=pd.read_parquet(path,columns=columns)
            combined=pd.concat([combined,parq])
        else:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-0{month}.parquet".format(year=y,month=x)
            parq= pd.read_parquet(path,columns=columns)
            combined=pd.concat([combined,parq])
        print(y, x)
    combined.to_parquet(r"/content/drive/MyDrive/Taxitaxi/Combined_Parquet/Pandas/yellow_tripdata_{year}".format(year=y))



print("fertig")


#All columns
#'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
# 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
# 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
# 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee'



In [ ]:
#I tried the same method, but with the Polars library.
#This turned out to be much more reliable, faster and less taxing on the hardware
#Polars is espescially made for larger data sets, which would exceed the available RAM

print("start")

columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime',  'trip_distance',
'PULocationID','DOLocationID','total_amount','tip_amount']

months=range(1,13)
years=range(2011,2023)

for y in years:
    for x in months:
        if x==1:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-0{month}.parquet".format(year=y,month=x)
            combined= pl.read_parquet(path,columns=columns).fill_nan(0)
        elif x>9:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-{month}.parquet".format(year=y,month=x)
            parq= pl.read_parquet(path,columns=columns).fill_nan(0)
            combined=pl.concat([combined,parq])
        else:
            path=r"/content/drive/MyDrive/Taxitaxi/By year/{year}/yellow_tripdata_{year}-0{month}.parquet".format(year=y,month=x)
            parq= pl.read_parquet(path,columns=columns).fill_nan(0)
            combined=pl.concat([combined,parq])
        print(y, x)
        print(len(combined))
    combined.write_parquet(r"/content/drive/MyDrive/Taxitaxi/Combined_Parquet/Polars/yellow_tripdata_{year}".format(year=y))


print("fertig")


# All Columns
#'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
# 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
# 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
# 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee'


